In [1]:
!pip install "transformers>4.45.2" datasets accelerate bitsandbytes peft trl


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig


device = "cuda" if torch.cuda.is_available() else "cpu"
base_model_path = "ibm-granite/granite-3.0-8b-instruct"
peft_model_path = "./granite-3.0-8b-instruct-finetuned-epoch3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.float16 # 미설정 시 느린 속도로 경고가 뜸
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_path)
model = AutoModelForCausalLM.from_pretrained(base_model_path, quantization_config=bnb_config, device_map="auto")

## special token 추가했을 경우 수행
# model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(model, peft_model_path)

model.eval()


config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

PeftModel(
  (base_model): LoraModel(
    (model): GraniteForCausalLM(
      (model): GraniteModel(
        (embed_tokens): Embedding(49155, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x GraniteDecoderLayer(
            (self_attn): GraniteAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

In [9]:
model = model.half() # 모델 속도 향상 FP16

In [17]:
input_text = "<|system|>\n한국어로 대답해주고, 반드시 존댓말로 대답해야해.\n<|user|>\n오늘 시험이 끝났어. 내일은 어떤 일들이 기다리고 있을까? 일반적으로 시험이 끝난 후에는 친구들 또는 가족과 만나 즐거운 시간을 즐긴다는 사실을 참고해서 답을해줘\n<|assistant|>"
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=250, do_sample=False)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|system|>
한국어로 대답해주고, 반드시 존댓말로 대답해야해.
<|user|>
오늘 시험이 끝났어. 내일은 어떤 일들이 기다리고 있을까? 일반적으로 시험이 끝난 후에는 친구들 또는 가족과 만나 즐거운 시간을 즐긴다는 사실을 참고해서 답을해줘
<|assistant|>시험이 끝난 후에는 일상적인 일들이 기다리고 있을 것입니다. 일반적으로는 친구들과 만나서 즐거운 시간을 보내거나, 가족과 함께 시간을 보내는 것도 좋은 방법입니다. 또한, 자신이 원하는 것을 할 수 있는 시간이 있을 때는 자신의 취미나 관심사를 즐길 수도 있습니다. 하지만, 시험이 끝난 후에는 충분한 휴식과 회복을 위해 충분한 시간을 보내는 것이 중요합니다.<|endoftext|> |endoftext|> |endoftext|> AI 모델의 답변입니다.<|endoftext|> |endoftext|> |endoftext|> AI 모델의 답변입니다.<|endoftext|> |endoftext|> AI 모델의 답변입니다.<|endoftext|> |endoftext|> AI 모델의 답변입니다.<|endoftext|> AI 모델의 답변입니다.<|endoftext|> AI 모델의 답변입니다.<|


In [6]:
next(model.parameters()).device

device(type='cuda', index=0)